In [2]:
import re
import time
import pickle
import numpy as np

from edit_distance import SequenceMatcher
import torch
from dataset import SpeechDataset
import matplotlib.pyplot as plt

from neural_decoder.dataset import getDatasetLoaders
import neural_decoder.lm_utils as lmDecoderUtils
from neural_decoder.model import GRUDecoder
import pickle
import argparse
import matplotlib.pyplot as plt
from neural_decoder.dataset import getDatasetLoaders
import neural_decoder.lm_utils as lmDecoderUtils
from neural_decoder.lm_utils import build_llama_1B
from neural_decoder.model import GRUDecoder
from neural_decoder.bit import BiT_Phoneme
import pickle
import argparse
from lm_utils import _cer_and_wer
import json
import os
import sys
import copy

/home3/ebrahim/miniconda3/envs/speech-bci/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def convert_sentence(s):
    s = s.lower()
    charMarks = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',
                 "'", ' ']
    ans = []
    for i in s:
        if(i in charMarks):
            ans.append(i)
    
    return ''.join(ans)

In [4]:
base_dir = "/home3/skaasyap/willett"

load_lm = True
# LM decoding hyperparameters
acoustic_scale = 0.8
blank_penalty = np.log(2)

run_for_llm = False

if run_for_llm:
    return_n_best = True
    rescore = False
    nbest = 100
    print("RUNNING IN LLM MODE")
else:
    return_n_best = False
    rescore = False
    nbest = 1
    print("RUNNING IN N-GRAM MODE")

RUNNING IN N-GRAM MODE


In [5]:
if load_lm and 'ngramDecoder' not in globals(): 
        
    lmDir = base_dir +'/lm/languageModel'
    ngramDecoder = lmDecoderUtils.build_lm_decoder(
        lmDir,
        acoustic_scale=acoustic_scale, #1.2
        nbest=nbest,
        beam=18
    )
    print("loaded LM")

loaded LM


I0728 20:11:06.811215 2117809 brain_speech_decoder.h:52] Reading fst /home3/skaasyap/willett/lm/languageModel/TLG.fst
I0728 20:14:10.093220 2117809 brain_speech_decoder.h:81] Reading symbol table /home3/skaasyap/willett/lm/languageModel/words.txt


In [9]:
# file path stuff for when running models on two servers (obiwan or leia)
# model storage locations and data path within model args differ depending on 
# the server. 
current_server = 'leia'
server_models_ran_on = 'obi'

if current_server == 'obi':
    
    base_path = '/data/willett_data/'
    
    if server_models_ran_on == 'obi':
        model_storage_path = '/data/willett_data/outputs/'
        
    elif server_models_ran_on == 'leia':
        model_storage_path = '/data/willett_data/leia_outputs/'

if current_server == 'leia':
    
    base_path = '/home3/ebrahim/save_transcripts/'
    
    if server_models_ran_on == 'obi':
        model_storage_path = '/home3/skaasyap/willett/obi_models/outputs/'
    
    elif server_models_ran_on == 'leia':
        model_storage_path = '/home3/skaasyap/willett/outputs/'
        
leia_to_obi_dict = {'/home3/skaasyap/willett/data_log_both':'/data/willett_data/ptDecoder_ctc_both', 
                    '/home3/skaasyap/willett/data':'/data/willett_data/ptDecoder_ctc',
                    '/home3/skaasyap/willett/data_log_both_held_out_days': '/data/willett_data/ptDecoder_ctc_both_held_out_days', 
                    '/home3/skaasyap/willett/data_log_both_held_out_days_1': '/data/willett_data/ptDecoder_ctc_both_held_out_days_1',
                    '/home3/skaasyap/willett/data_log_both_held_out_days_2': '/data/willett_data/ptDecoder_ctc_both_held_out_days_2'}

obi_to_leia_dict = {v:k for k, v in leia_to_obi_dict.items()}


In [11]:
device = "cuda:2"

models_to_run = ['neurips_gru_data_log_time_masked_lr_schedule']

shared_output_file = ''

if len(shared_output_file) > 0:
    print("Writing to shared output file")
    write_mode = "a"
else:
    write_mode = "w"
    
seeds_list = [0]
partition = "test" # "test"
run_lm = True

comp_on_reduced = False # if running restricted days analyses
fill_max_day = False
skip_days = [[]]

day_edit_distance = 0
day_seq_length = 0
prev_day = None

if partition == 'test':
    saveFolder_transcripts = f"{base_path}model_transcriptions/"
else:
    saveFolder_transcripts = f"{base_path}model_transcriptions_retest/"
    
day_cer_dict = {}
total_wer_dict = {}

for seed in seeds_list:
    
    day_cer_dict[seed] = []
    total_wer_dict[seed] = []
            
    for mn, model_name_str in enumerate(models_to_run):
        
        effective_length_arr = []
     
        modelPath = f"{model_storage_path}{model_name_str}_seed_{seed}"
        
        if len(shared_output_file) > 0:
            output_file = f"{shared_output_file}_seed_{seed}"
            print(output_file)
        else:
            output_file = f"{model_name_str}_seed_{seed}"
            
        print(f"Running model: {model_name_str}_seed_{seed}")
            
        with open(modelPath + "/args", "rb") as handle:
            args = pickle.load(handle)
            
        # modify data file path based on which server model was ran on 
        # and which server is currently being used to run this notebok.
        if server_models_ran_on != current_server:
            
            if current_server == 'leia':
                data_file = obi_to_leia_dict[args['datasetPath']]
                
            elif current_server == 'obi':
                data_file = leia_to_obi_dict[args['datasetPath']]
                
        else:

            data_file = args['datasetPath']
            
        print("DATA FILE: ", data_file)
      
        if 'ventral_6v_only' not in args:
            args['ventral_6v_only'] = False
            
        trainLoaders, testLoaders, loadedData = getDatasetLoaders(
            data_file, 8, args['restricted_days'], 
            args['ventral_6v_only']
        )
        
        # if true, model is a GRU
        if 'nInputFeatures' in args.keys():
            
            if 'max_mask_pct' not in args:
                args['max_mask_pct'] = 0
            if 'num_masks' not in args:
                args['num_masks'] = 0
            if 'input_dropout' not in args:
                args['input_dropout'] = 0
            if 'linderman_lab' not in args:
                args['linderman_lab'] = False
                
            print("Loading GRU")
            model = GRUDecoder(
                neural_dim=args["nInputFeatures"],
                n_classes=args["nClasses"],
                hidden_dim=args["nUnits"],
                layer_dim=args["nLayers"],
                nDays=args['nDays'],
                dropout=args["dropout"],
                device=device,
                strideLen=args["strideLen"],
                kernelLen=args["kernelLen"],
                gaussianSmoothWidth=args["gaussianSmoothWidth"],
                bidirectional=args["bidirectional"],
                input_dropout=args['input_dropout'], 
                max_mask_pct=args['max_mask_pct'],
                num_masks=args['num_masks'], 
                linderman_lab=args['linderman_lab']
            ).to(device)

        else:
            
            if 'mask_token_zero' not in args:
                args['mask_token_zero'] = False
                            
            # Instantiate model
            # set training relevant parameters for MEMO, doesn't matter for other runs because they are 
            # only run in eval mode.
            model = BiT_Phoneme(
                patch_size=args['patch_size'],
                dim=args['dim'],
                dim_head=args['dim_head'],
                nClasses=args['nClasses'],
                depth=args['depth'],
                heads=args['heads'],
                mlp_dim_ratio=args['mlp_dim_ratio'],
                dropout=0,
                input_dropout=0,
                gaussianSmoothWidth=args['gaussianSmoothWidth'],
                T5_style_pos=args['T5_style_pos'],
                max_mask_pct=0.0,
                num_masks=0, 
                mask_token_zeros=args['mask_token_zero'], 
                num_masks_channels=0, 
                max_mask_channels=0, 
                dist_dict_path=0
            ).to(device)
            
            
        ckpt_path = modelPath + '/modelWeights'
        model.load_state_dict(torch.load(ckpt_path, map_location=device), strict=True)
        model = model.to(device)
        
        model.eval()

        model_outputs = {
            "logits": [],
            "logitLengths": [],
            "trueSeqs": [],
            "transcriptions": [],
        }
        
        total_edit_distance = 0
        total_seq_length = 0

        if partition == "competition":
        
            testDayIdxs = [4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18, 19, 20]
                
        elif partition == "test":
            
            testDayIdxs = range(len(loadedData[partition])) 
            
        ground_truth_sentences = []
        
        #print("RESTRICTED DAYS: ", args['restricted_days'])
        
        for i, testDayIdx in enumerate(testDayIdxs):
            
            
            day_outputs = {
                "logits": [],
                "logitLengths": [],
                "trueSeqs": [],
                "transcriptions": [],
            }
            
            #if len(skip_days[mn]) > 0:
            #    if testDayIdx in skip_days[mn]:
            #        print("SKIPPING DAY: ", testDayIdx)
            #        continue
            
            if len(args['restricted_days']) > 0:
                if testDayIdx not in args['restricted_days']:
                    continue
                
            test_ds = SpeechDataset([loadedData[partition][i]])
            
            test_loader = torch.utils.data.DataLoader(
                test_ds, batch_size=1, shuffle=False, num_workers=0
            )
            
            for j, (X, y, X_len, y_len, _) in enumerate(test_loader):
                        
                X, y, X_len, y_len, dayIdx = (
                    X.to(device),
                    y.to(device),
                    X_len.to(device),
                    y_len.to(device),
                    torch.tensor([testDayIdx], dtype=torch.int64).to(device),
                )
                
                if args['ventral_6v_only']:
                    X = X[:, :, :128]
                
                if fill_max_day:
                    dayIdx.fill_(args['maxDay'])
                      
                with torch.no_grad():
                    
                    pred = model.forward(X, X_len, dayIdx)
                
                if hasattr(model, 'compute_length'):
                    adjustedLens = model.compute_length(X_len)
                else:
                    adjustedLens = ((X_len - model.kernelLen) / model.strideLen).to(torch.int32)
                    
                for iterIdx in range(pred.shape[0]):
                    
                    trueSeq = np.array(y[iterIdx][0 : y_len[iterIdx]].cpu().detach())
                    model_outputs["logits"].append(pred[iterIdx].cpu().detach().numpy())
                    
                    model_outputs["logitLengths"].append(
                        adjustedLens[iterIdx].cpu().detach().item()
                    )
                    
                    model_outputs["trueSeqs"].append(trueSeq)
                    
                    decodedSeq = torch.argmax(
                        torch.tensor(pred[iterIdx, 0 : adjustedLens[iterIdx], :]),
                        dim=-1,
                    ) 
                    
                    decodedSeq = torch.unique_consecutive(decodedSeq, dim=-1)
                    decodedSeq = decodedSeq.cpu().detach().numpy()
                    decodedSeq = np.array([i for i in decodedSeq if i != 0])
                    
                    matcher = SequenceMatcher(
                        a=trueSeq.tolist(), b=decodedSeq.tolist()
                    )
                    
                    total_edit_distance += matcher.distance()
                    total_seq_length += len(trueSeq)
                    
                    day_edit_distance += matcher.distance()
                    day_seq_length += len(trueSeq)
                    
                transcript = loadedData[partition][i]["transcriptions"][j].strip()
                transcript = re.sub(r"[^a-zA-Z\- \']", "", transcript)
                transcript = transcript.replace("--", "").lower()
                model_outputs["transcriptions"].append(transcript)
                
            cer_day = day_edit_distance / day_seq_length
            day_cer_dict[seed].append(cer_day)
            print("CER DAY: ", cer_day)
            day_edit_distance = 0 
            day_seq_length = 0

        cer = total_edit_distance / total_seq_length
        
        print("Model performance: ", cer)
        
        if run_lm:
            
            print("Running n-gram LM")
            
            llm_outputs = []
            start_t = time.time()
            nbest_outputs = []
            
            sample_times = []

            for j in range(len(model_outputs["logits"])):
                logits = model_outputs["logits"][j]

                # Move blank token to the end
                logits = np.concatenate([logits[:, 1:], logits[:, 0:1]], axis=-1)

                # Rearrange logits for decoding
                logits = lmDecoderUtils.rearrange_speech_logits(logits[None, :, :], has_sil=True)

                # Time individual sample decoding
                sample_start = time.time()
                nbest = lmDecoderUtils.lm_decode(
                    ngramDecoder,
                    logits[0],
                    blankPenalty=blank_penalty,
                    returnNBest=return_n_best,
                    rescore=rescore,
                )
                sample_end = time.time()

                sample_times.append(sample_end - sample_start)
                nbest_outputs.append(nbest)

            # Convert to numpy array for stats
            sample_times = np.array(sample_times)
            mean_time = np.mean(sample_times)
            std_time = np.std(sample_times)

            print(f"Average decoding time per sample: {mean_time:.3f} seconds")
            print(f"Standard deviation: {std_time:.3f} seconds")
 
            if run_for_llm:
                
                print("SAVING OUTPUTS FOR LLM")
                with open(f"{saveFolder_transcripts}{model_name_str}_seed_{seed}_model_outputs.pkl", "wb") as f:
                    pickle.dump(model_outputs, f)
                    
                with open(f"{saveFolder_transcripts}{model_name_str}_seed_{seed}_nbest.pkl", "wb") as f:
                    pickle.dump(nbest_outputs, f)
                
            else:
                
                # just get perf with greedy decoding
                for i in range(len(model_outputs["transcriptions"])):
                    model_outputs["transcriptions"][i] = model_outputs["transcriptions"][i].strip()
                    nbest_outputs[i] = nbest_outputs[i].strip()
                
                # lower case + remove puncs
                for i in range(len(model_outputs["transcriptions"])):
                    model_outputs["transcriptions"][i] = convert_sentence(model_outputs["transcriptions"][i])

                cer, wer = _cer_and_wer(nbest_outputs, model_outputs["transcriptions"], 
                                    outputType='speech', returnCI=True)

                print("CER and WER after 3-gram LM: ", cer, wer)       
                
                out_file = os.path.join(saveFolder_transcripts, output_file)   # no extension per your spec
                
                with open(out_file + '.txt', write_mode, encoding="utf-8") as f:
                    f.write("\n".join(nbest_outputs)+ "\n")   # one line per LLM output  
                    
                total_wer_dict[seed] = wer
                
#if memo:
#    with open(f"/data/willett_data/paper_results/{model_name_str}_per_memo.pkl", "wb") as f:
#        pickle.dump(day_cer_dict, f)
        
#    with open(f"/data/willett_data/paper_results/{model_name_str}_wer_memo.pkl", "wb") as f:
#        pickle.dump(total_wer_dict, f)ls
        
#else:
#    
#    with open(f"/data/willett_data/paper_results/{model_name_str}_per.pkl", "wb") as f:
#        pickle.dump(day_cer_dict, f)
#        
#    with open(f"/data/willett_data/paper_results/{model_name_str}_wer.pkl", "wb") as f:
#        pickle.dump(total_wer_dict, f)


Running model: neurips_gru_data_log_time_masked_lr_schedule_seed_0
DATA FILE:  /home3/skaasyap/willett/data_log_both
Loading GRU


/home3/ebrahim/miniconda3/envs/speech-bci/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home3/ebrahim/neural_seq_decoder/src/neural_decoder/augmentations.py:170: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:895.)
  return self.conv(input, weight=self.weight, groups=self.groups, padding="same")
/tmp/ipykernel_2117809/3127162098.py:229: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(pred[iterIdx, 0 : adjustedLens[iterIdx], :]),


CER DAY:  0.3053691275167785
CER DAY:  0.21379310344827587
CER DAY:  0.19488817891373802
CER DAY:  0.27472527472527475
CER DAY:  0.11569506726457399
CER DAY:  0.12052730696798493
CER DAY:  0.12771996215704826
CER DAY:  0.17557932263814616
CER DAY:  0.14183266932270916
CER DAY:  0.18333333333333332
CER DAY:  0.17672790901137359
CER DAY:  0.20128676470588236
CER DAY:  0.17565055762081785
CER DAY:  0.13137032842582105
CER DAY:  0.1159678858162355
CER DAY:  0.11732851985559567
CER DAY:  0.14074074074074075
CER DAY:  0.17769002961500494
CER DAY:  0.15077605321507762
CER DAY:  0.12043435340572557
CER DAY:  0.10753768844221105
CER DAY:  0.15182755388940955
CER DAY:  0.15013901760889714
CER DAY:  0.178125
Model performance:  0.16103338698361602
Running n-gram LM
Average decoding time per sample: 0.033 seconds
Standard deviation: 0.044 seconds
CER and WER after 3-gram LM:  (0.11767696945483601, 0.10853764289293968, 0.12687042276706295) (0.17366793962538643, 0.1602196028287572, 0.187295855244327

In [ ]:
#  GRU SHORT NON OVERLAPPING 4 4 1024 units
(0.18748863429714493, 0.1733481163205817, 0.20224525762022216)

#  GRU SHORT NON OVERLAPPING 4 4 768 units

#  GRU SHORT NON OVERLAPPING 4 4 512 units
CER and WER after 3-gram LM:  (0.14898485453819055, 0.13837987803968166, 0.15996022357119136) (0.20258228768867068, 0.18814850882093556, 0.21746997243144325)
CER and WER after 3-gram LM:  (0.15410598191261396, 0.1433691371958222, 0.16531221010578537) (0.2078559738134206, 0.19354253159267176, 0.22291510680325294)
CER and WER after 3-gram LM:  (0.15980823012385137, 0.14892442509510415, 0.17068014782871976) (0.21640298236042918, 0.20112935287334593, 0.23179312681415)

# GRU SHORT 4 layers 
(0.18057828696126568, 0.16678629445964432, 0.1944448216548977)
(0.18803418803418803, 0.1738094510763848, 0.20240924226959411
(0.18003273322422259, 0.16660588574119756, 0.19369812058380678)
(0.1867612293144208, 0.1725848132153764, 0.2012581255961117)



# GRU OVERLAPPING 1024 Units
(0.12759234373297498, 0.1177953831800847, 0.13754510964858174) (0.17821422076741225, 0.16463861375291677, 0.1922452709629667)
(0.12494097991501107, 0.11526411112142802, 0.13481354564910503) (0.17585015457355882, 0.16250681574211354, 0.1895082789745812)
(0.12839138488359422, 0.1185083430244113, 0.13813175907892178) (0.18221494817239497, 0.16842248424125197, 0.19599519410498778)
(0.1250862601242146, 0.11504058168559311, 0.13520741295347288) (0.1778505182760502, 0.16381152203794247, 0.19241147368812195)


In [ ]:
# TM TRANSFORMER DEPTH 5
CER and WER after 3-gram LM:  (0.115933606944394, 0.1067014506708351, 0.1252127983640603) (0.16657574104382614, 0.15368177508624756, 0.1795666612468941)
CER and WER after 3-gram LM:  (0.12352449787527695, 0.11433927553171039, 0.1328864642207172) (0.1783960720130933, 0.1655529013791415, 0.19171298671996517)
CER and WER after 3-gram LM:  (0.11909345149457015, 0.10942212714998742, 0.12876255652512378) (0.1691216584833606, 0.15584404058844953, 0.18245120621353922)
CER and WER after 3-gram LM:  (0.11691424835651763, 0.10748297572304032, 0.12684070407993206) (0.16930350972904165, 0.15640483471072475, 0.1827764567863795)

# TM TRANSFORMER DEPTH 7
CER and WER after 3-gram LM:  (0.11273744234191697, 0.10379952109750006, 0.12199860729002777) (0.1691216584833606, 0.15625540724491016, 0.18251230526392123)
CER and WER after 3-gram LM:  (0.12334289761377257, 0.11391797673479954, 0.13266513338160046) (0.17966903073286053, 0.166331520250247, 0.19331443798449613)
CER and WER after 3-gram LM:  (0.1137180837540406, 0.10443577552864726, 0.1234524891407505) (0.1629387161302055, 0.1501740988670145, 0.17593611199732556)
CER and WER after 3-gram LM:  (0.1206552137435078, 0.11126393883529243, 0.13027161823251315) (0.17421349336242953, 0.160873204413609, 0.18752321917567022)

In [24]:
print("DEPTH 5: ", (16.665 + 17.839 + 16.912 + 16.930)/4)
print("DEPTH 7: ", (16.9121+ 17.966 + 16.2938 + 17.41)/4)


DEPTH 5:  17.0865
DEPTH 7:  17.145475


In [ ]:
args['restricted_days']

In [ ]:
args['ventral_6v_only']


In [4]:
import pandas as pd
import numpy as np

In [19]:
def print_mean_and_sem(file_name):
    data_dict = pd.read_pickle(f'/home3/ebrahim/paper_results_wer/{file_name}.pkl')
    averaged_dict = np.mean(np.array(list(data_dict.values())), axis=0)
    std_dict = np.std(np.array(list(data_dict.values())), axis=0)
    print("FILENAME: ", file_name)
    print("MEAN: ", averaged_dict*100)
    print("SEM: ", std_dict*100/np.sqrt(4))

In [20]:
print_mean_and_sem('transformer_shortened_held_out_days_big_diet64corp')
print_mean_and_sem('transformer_shortened_held_out_days_big_baseline')

FILENAME:  transformer_shortened_held_out_days_big_diet64corp
MEAN:  [26.97180221 26.37997433 27.56913303 30.45539033 29.70240366 29.08466394
 31.35994587 31.91169672]
SEM:  [0.27679234 0.295103   0.12629396 0.40539519 0.20479693 0.38262064
 0.75516775 1.25215305]
FILENAME:  transformer_shortened_held_out_days_big_baseline
MEAN:  [28.87208827 34.49935815 44.78699552 53.28996283 59.97710797 52.71073153
 62.95669824 66.47018031]
SEM:  [0.47007426 0.43882473 0.33980817 0.24676428 0.31259003 0.51828908
 0.40481692 0.57613577]


In [22]:
print_mean_and_sem('transformer_short_held_out_normal_diet64corp1e3')
print_mean_and_sem('transformer_short_held_out_normal_baseline')

FILENAME:  transformer_short_held_out_normal_diet64corp1e3
MEAN:  [21.14312268 21.23235406 21.39806907 21.93279206 22.26074896]
SEM:  [0.31526679 0.20612536 0.22255797 0.65391546 0.60046117]
FILENAME:  transformer_short_held_out_normal_baseline
MEAN:  [22.74163569 25.47691721 25.42703305 30.1308074  32.58206195]
SEM:  [0.26101016 0.27886406 0.32857602 0.33030714 0.51801989]
